# Data for Fig. 1(g-i): Eigenspectra of the Liouvillian (left panel figures)

In [1]:
####### execute using "python m.py"
####### Created: 17/05/2024 #########
####### Modified: 21/05/2024 #########
##################
################## Bosonic ME projected ino a sector N=N_a+N_b and also into
################## two sectors with different N.
################## It compares with the eigenvalues for the BTC using qutip
##################

import time
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse import linalg
import matplotlib.pyplot as plt
import qutip as qt


##########################################
############## $$$ MY FUNCTIONS $$$ ######
##########################################

################### This function creates the Liouvillian projected into a N sector
################### It also provides useful operators.
def bosonic_liouvillian_setup(N,kap,U,w,n_th):

    format1="lil" ### SPARSE MATRIX: format=csc is the same as MATLAB
    format2="csc" ### format=csc/csr is good for calculations, but
                  ### "lil" is much much faster for preparing the array

    dim_Hilbert=(N+1)**2 ## We square because we build the Liouvillian

    ######### We initialize the sparse matrices that we need
    liouv_op=0*sparse.eye(dim_Hilbert,dtype=complex,format=format1)
    a_op=0*sparse.eye(dim_Hilbert,format=format1)
    b_op=0*sparse.eye(dim_Hilbert,format=format1)
    bda_op=0*sparse.eye(dim_Hilbert,format=format1)
    na_op=0*sparse.eye(dim_Hilbert,format=format1)
    nb_op=0*sparse.eye(dim_Hilbert,format=format1)

    if format2 == "csc":
        flat_state=sparse.csc_matrix(np.zeros((dim_Hilbert,1)))
    if format2 == "csr":
        flat_state=sparse.csr_matrix(np.zeros((dim_Hilbert,1)))


    ######## The following vectors are used to keep track of the quantum numbers
    vA_r=np.zeros(dim_Hilbert,dtype=int)  ### Matrix elements become
    vB_r=np.zeros(dim_Hilbert,dtype=int)  ### vector elements in the doubled space
    vA_l=np.zeros(dim_Hilbert,dtype=int)  ### |Na_r,Nb_r><Na_l,Nb_l|
    vB_l=np.zeros(dim_Hilbert,dtype=int)  ### --> |Na_r,Nb_r;Na_l,Nb_l>>

    #### We use a dictionary to store the labels of the states |Na_r,Nb_r;Na_l,Nb_l>>
    labels={}

    k=0
    for na_r in range(0,N+1):
        nb_r=N-na_r
        for na_l in range(0,N+1):
            nb_l=N-na_l
            vA_r[k]=na_r
            vB_r[k]=nb_r
            vA_l[k]=na_l
            vB_l[k]=nb_l
            key=str(na_r)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
            #print(key)
            labels[key]=k
            k=k+1

    #print(vA_r)
    ###### We proceed to create the Liouvillian matrix and operators
    for k2 in range(0,dim_Hilbert): ########### CAREFUL INDICES: k2 is the receiver, k1 is the origin.

        na_r=vA_r[k2]
        nb_r=vB_r[k2]
        na_l=vA_l[k2]
        nb_l=vB_l[k2]

        ###################### First we create the Liouvillian term by term
        ######################
        ####### Exchange term:
        key1=str(na_r-1)+","+str(nb_r+1)+","+str(na_l)+","+str(nb_l)
        if key1 in labels:
            k1=labels[key1]  ## Identify the state to which we are coupling
            liouv_op[k2,k1]+=-0.5j*w*np.sqrt((na_r)*(nb_r+1)) ## Set up the matrix element

        key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l)+","+str(nb_l)
        if key1 in labels:
            k1=labels[key1]
            liouv_op[k2,k1]+=-0.5j*w*np.sqrt((na_r+1)*(nb_r))

        key1=str(na_r)+","+str(nb_r)+","+str(na_l+1)+","+str(nb_l-1)
        if key1 in labels:
            k1=labels[key1]
            liouv_op[k2,k1]+=+0.5j*w*np.sqrt((na_l+1)*(nb_l))

        key1=str(na_r)+","+str(nb_r)+","+str(na_l-1)+","+str(nb_l+1)
        if key1 in labels:
            k1=labels[key1]
            liouv_op[k2,k1]+=+0.5j*w*np.sqrt((na_l)*(nb_l+1))

        ######################
        ####### Interaction term:
        ####### This term is diagonal, so it is easy:
        liouv_op[k2,k2]+=-1j*(U/(N/2))*(na_r*(na_r-1)+nb_r*(nb_r-1)-na_l*(na_l-1)-nb_l*(nb_l-1))

        ######################
        ####### First dissipator: (the one proportional to n_th+1)
        key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l+1)+","+str(nb_l-1)
        if key1 in labels:
            k1=labels[key1]
            liouv_op[k2,k1]+=(1+n_th)*(kap/(N/2))*np.sqrt((na_r+1)*nb_r*(na_l+1)*nb_l)

        liouv_op[k2,k2]+=-0.5*(1+n_th)*(kap/(N/2))*(na_r*(nb_r+1)+na_l*(nb_l+1))

        ######################
        ####### Second dissipator: (the one proportional to n_th)
        key1=str(na_r-1)+","+str(nb_r+1)+","+str(na_l-1)+","+str(nb_l+1)
        if key1 in labels:
            k1=labels[key1]
            liouv_op[k2,k1]+=(n_th)*(kap/(N/2))*np.sqrt((nb_r+1)*na_r*(nb_l+1)*na_l)

        liouv_op[k2,k2]+=-0.5*(n_th)*(kap/(N/2))*(nb_r*(na_r+1)+nb_l*(na_l+1))


        ###################### We now create some useful operators:
        ######################

        ####### na_op|Na_r,Nb_r><Na_l,Nb_l|=Na_r|Na_r,Nb_r><Na_l,Nb_l|
        na_op[k2,k2]+=na_r

        ####### nb_op|Na_r,Nb_r><Na_l,Nb_l|=Nb_r|Na_r,Nb_r><Na_l,Nb_l|
        nb_op[k2,k2]+=nb_r

        ####### a_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Na_r)|Na_r-1,Nb_r><Na_l,Nb_l|
        key1=str(na_r+1)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
        if key1 in labels:
            k1=labels[key1]
            a_op[k2,k1]+=np.sqrt(na_r+1)

        ####### b_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Nb_r)|Na_r,Nb_r-1><Na_l,Nb_l|
        key1=str(na_r)+","+str(nb_r+1)+","+str(na_l)+","+str(nb_l)
        if key1 in labels:
            k1=labels[key1]
            b_op[k2,k1]+=np.sqrt(nb_r+1)

        ####### bda_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Na_r*(Nb_r+1))|Na_r-1,Nb_r+1><Na_l,Nb_l|
        key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l)+","+str(nb_l)
        if key1 in labels:
            k1=labels[key1]
            bda_op[k2,k1]+=np.sqrt((na_r+1)*nb_r)

        ####### Flat state: this is used to perform the trace in the vectorized formalism

        if na_r == na_l:
            if nb_r == nb_l:
                flat_state[k2]=1.0


    ####################### We now convert the matrices to csc/csr format:
    if format2=='csc':
        liouv_op=sparse.lil_matrix.tocsc(liouv_op)
        a_op=sparse.lil_matrix.tocsc(a_op)
        b_op=sparse.lil_matrix.tocsc(b_op)
        na_op=sparse.lil_matrix.tocsc(na_op)
        nb_op=sparse.lil_matrix.tocsc(nb_op)
        bda_op=sparse.lil_matrix.tocsc(bda_op)
    elif format2=='csr':
        liouv_op=sparse.lil_matrix.tocsr(liouv_op)
        a_op=sparse.lil_matrix.tocsr(a_op)
        b_op=sparse.lil_matrix.tocsr(b_op)
        na_op=sparse.lil_matrix.tocsr(na_op)
        nb_op=sparse.lil_matrix.tocsr(nb_op)
        bda_op=sparse.lil_matrix.tocsr(bda_op)


    return liouv_op,flat_state,a_op,b_op,bda_op,na_op,nb_op

################### This function creates the Liouvillian
################### projected into two N sectors and
################### their coherences (N1,N1), (N1,N2), (N2,N1), (N2,N2)
################### It also provides useful operators.
def bosonic_liouvillian_2sectors_setup(N1,N2,kap,U,w,n_th):
	format1="lil" ### SPARSE MATRIX: format=csc is the same as MATLAB
	format2="csc" ### format=csc/csr is good for calculations, but
		          ### "lil" is much much faster for preparing the array

	dim_Hilbert=(N1+1+N2+1)**2 ## We square because we build the Liouvillian

	kap_S=kap/(N1/2)   ###### IMPORTANT:
	U_S=U/(N1/2)       ###### One needs to choose by which sector one renormalizes the parameters


	######### We initialize the sparse matrices that we need
	liouv_op=0*sparse.eye(dim_Hilbert,dtype=complex,format=format1)
	a_op=0*sparse.eye(dim_Hilbert,format=format1)
	b_op=0*sparse.eye(dim_Hilbert,format=format1)
	ad_op=0*sparse.eye(dim_Hilbert,format=format1)
	bd_op=0*sparse.eye(dim_Hilbert,format=format1)
	bda_op=0*sparse.eye(dim_Hilbert,format=format1)
	na_op=0*sparse.eye(dim_Hilbert,format=format1)
	nb_op=0*sparse.eye(dim_Hilbert,format=format1)

	if format2 == "csc":
		flat_state=sparse.csc_matrix(np.zeros((dim_Hilbert,1)))
	if format2 == "csr":
		flat_state=sparse.csr_matrix(np.zeros((dim_Hilbert,1)))


	######## The following vectors are used to keep track of the quantum numbers
	vA_r=np.zeros(dim_Hilbert,dtype=int)  ### Matrix elements become
	vB_r=np.zeros(dim_Hilbert,dtype=int)  ### vector elements in the doubled space
	vA_l=np.zeros(dim_Hilbert,dtype=int)  ### |Na_r,Nb_r><Na_l,Nb_l|
	vB_l=np.zeros(dim_Hilbert,dtype=int)  ### --> |Na_r,Nb_r;Na_l,Nb_l>>

	#### We use a dictionary to store the labels of the states |Na_r,Nb_r;Na_l,Nb_l>>
	labels={}

	k=0
	###### We got four general catergories for |Na_r,Nb_r;Na_l,Nb_l>>
	for na_r in range(0,N1+1):
		nb_r=N1-na_r
		for na_l in range(0,N1+1): #### The first is Na_r+Nb_r=N1; Na_l+Nb_l=N1
		    nb_l=N1-na_l
		    vA_r[k]=na_r
		    vB_r[k]=nb_r
		    vA_l[k]=na_l
		    vB_l[k]=nb_l
		    key=str(na_r)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		    #print(key)
		    labels[key]=k
		    k=k+1

		for na_l in range(0,N2+1): #### The second is Na_r+Nb_r=N1; Na_l+Nb_l=N2
		    nb_l=N2-na_l
		    vA_r[k]=na_r
		    vB_r[k]=nb_r
		    vA_l[k]=na_l
		    vB_l[k]=nb_l
		    key=str(na_r)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		    #print(key)
		    labels[key]=k
		    k=k+1

	for na_r in range(0,N2+1):
		nb_r=N2-na_r
		for na_l in range(0,N1+1): #### The third is Na_r+Nb_r=N2; Na_l+Nb_l=N1
		    nb_l=N1-na_l
		    vA_r[k]=na_r
		    vB_r[k]=nb_r
		    vA_l[k]=na_l
		    vB_l[k]=nb_l
		    key=str(na_r)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		    #print(key)
		    labels[key]=k
		    k=k+1

		for na_l in range(0,N2+1): #### The fourth is Na_r+Nb_r=N2; Na_l+Nb_l=N2
		    nb_l=N2-na_l
		    vA_r[k]=na_r
		    vB_r[k]=nb_r
		    vA_l[k]=na_l
		    vB_l[k]=nb_l
		    key=str(na_r)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		    #print(key)
		    labels[key]=k
		    k=k+1



	#print(vA_r)
	###### We proceed to create the Liouvillian matrix and operators
	for k2 in range(0,dim_Hilbert): ########### CAREFUL INDICES: k2 is the receiver, k1 is the origin.


		na_r=vA_r[k2]
		nb_r=vB_r[k2]
		na_l=vA_l[k2]
		nb_l=vB_l[k2]

		###################### First we create the Liouvillian term by term
		######################
		####### Exchange term:
		key1=str(na_r-1)+","+str(nb_r+1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]  ## Identify the state to which we are coupling
		    liouv_op[k2,k1]+=-0.5j*w*np.sqrt((na_r)*(nb_r+1)) ## Set up the matrix element

		key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=-0.5j*w*np.sqrt((na_r+1)*(nb_r))

		key1=str(na_r)+","+str(nb_r)+","+str(na_l+1)+","+str(nb_l-1)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=+0.5j*w*np.sqrt((na_l+1)*(nb_l))

		key1=str(na_r)+","+str(nb_r)+","+str(na_l-1)+","+str(nb_l+1)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=+0.5j*w*np.sqrt((na_l)*(nb_l+1))

		######################
		####### Interaction term:
		####### This term is diagonal, so it is easy:
		liouv_op[k2,k2]+=-1j*(U_S)*(na_r*(na_r-1)+nb_r*(nb_r-1))  #####
		liouv_op[k2,k2]+=+1j*(U_S)*(na_l*(na_l-1)+nb_l*(nb_l-1))

		######################
		####### First dissipator: (the one proportional to n_th+1)
		key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l+1)+","+str(nb_l-1)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=(1+n_th)*(kap_S)*np.sqrt((na_r+1)*nb_r*(na_l+1)*nb_l)

		liouv_op[k2,k2]+=-0.5*(1+n_th)*(kap_S)*(na_r*(nb_r+1))
		liouv_op[k2,k2]+=-0.5*(1+n_th)*(kap_S)*(na_l*(nb_l+1))

		######################
		####### Second dissipator: (the one proportional to n_th)
		key1=str(na_r-1)+","+str(nb_r+1)+","+str(na_l-1)+","+str(nb_l+1)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=(n_th)*(kap_S)*np.sqrt((nb_r+1)*na_r*(nb_l+1)*na_l)

		liouv_op[k2,k2]+=-0.5*(n_th)*(kap_S)*(nb_r*(na_r+1))
		liouv_op[k2,k2]+=-0.5*(n_th)*(kap_S)*(nb_l*(na_l+1))


		###################### We now create some useful operators:
		######################

		####### na_op|Na_r,Nb_r><Na_l,Nb_l|=Na_r|Na_r,Nb_r><Na_l,Nb_l|
		na_op[k2,k2]+=na_r

		####### nb_op|Na_r,Nb_r><Na_l,Nb_l|=Nb_r|Na_r,Nb_r><Na_l,Nb_l|
		nb_op[k2,k2]+=nb_r

		####### a_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Na_r)|Na_r-1,Nb_r><Na_l,Nb_l|
		key1=str(na_r+1)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    a_op[k2,k1]+=np.sqrt(na_r+1)

		####### b_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Nb_r)|Na_r,Nb_r-1><Na_l,Nb_l|
		key1=str(na_r)+","+str(nb_r+1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    b_op[k2,k1]+=np.sqrt(nb_r+1)

		####### ad_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Na_r+1)|Na_r+1,Nb_r><Na_l,Nb_l|
		key1=str(na_r-1)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    ad_op[k2,k1]+=np.sqrt(na_r)

		####### bd_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Nb_r+1)|Na_r,Nb_r+1><Na_l,Nb_l|
		key1=str(na_r)+","+str(nb_r-1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    bd_op[k2,k1]+=np.sqrt(nb_r)

		####### bda_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Na_r*(Nb_r+1))|Na_r-1,Nb_r+1><Na_l,Nb_l|
		key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    bda_op[k2,k1]+=np.sqrt((na_r+1)*nb_r)

		####### Flat state: this is used to perform the trace in the vectorized formalism

		if na_r == na_l:
			if nb_r == nb_l:
				flat_state[k2]=1.0


	####################### We now convert the matrices to csc/csr format:
	if format2=='csc':
		liouv_op=sparse.lil_matrix.tocsc(liouv_op)
		a_op=sparse.lil_matrix.tocsc(a_op)
		b_op=sparse.lil_matrix.tocsc(b_op)
		ad_op=sparse.lil_matrix.tocsc(ad_op)
		bd_op=sparse.lil_matrix.tocsc(bd_op)
		na_op=sparse.lil_matrix.tocsc(na_op)
		nb_op=sparse.lil_matrix.tocsc(nb_op)
		bda_op=sparse.lil_matrix.tocsc(bda_op)
	elif format2=='csr':
		liouv_op=sparse.lil_matrix.tocsr(liouv_op)
		a_op=sparse.lil_matrix.tocsr(a_op)
		b_op=sparse.lil_matrix.tocsr(b_op)
		ad_op=sparse.lil_matrix.tocsr(ad_op)
		bd_op=sparse.lil_matrix.tocsr(bd_op)
		na_op=sparse.lil_matrix.tocsr(na_op)
		nb_op=sparse.lil_matrix.tocsr(nb_op)
		bda_op=sparse.lil_matrix.tocsr(bda_op)


	return liouv_op,flat_state,a_op,b_op,ad_op,bd_op,bda_op,na_op,nb_op


################### This function creates the Liouvillian block corresponding to
################### **just** the coherences between sectors N1 and N2: (N1,N2), (N2,N1)
################### It also provides useful operators.
def bosonic_liouvillian_coherences2sectors_setup(N1,N2,kap,U,w,n_th):
	format1="lil" ### SPARSE MATRIX: format=csc is the same as MATLAB
	format2="csc" ### format=csc/csr is good for calculations, but
		          ### "lil" is much much faster for preparing the array

	dim_Hilbert=2*(N1+1)*(N2+1) ## We square because we build the Liouvillian

	kap_S=kap/(N1/2)   ###### IMPORTANT:
	U_S=U/(N1/2)       ###### One needs to choose by which sector one renormalizes the parameters


	######### We initialize the sparse matrices that we need
	liouv_op=0*sparse.eye(dim_Hilbert,dtype=complex,format=format1)
	a_op=0*sparse.eye(dim_Hilbert,format=format1)
	b_op=0*sparse.eye(dim_Hilbert,format=format1)
	bda_op=0*sparse.eye(dim_Hilbert,format=format1)
	na_op=0*sparse.eye(dim_Hilbert,format=format1)
	nb_op=0*sparse.eye(dim_Hilbert,format=format1)

	if format2 == "csc":
		flat_state=sparse.csc_matrix(np.zeros((dim_Hilbert,1)))
	if format2 == "csr":
		flat_state=sparse.csr_matrix(np.zeros((dim_Hilbert,1)))


	######## The following vectors are used to keep track of the quantum numbers
	vA_r=np.zeros(dim_Hilbert,dtype=int)  ### Matrix elements become
	vB_r=np.zeros(dim_Hilbert,dtype=int)  ### vector elements in the doubled space
	vA_l=np.zeros(dim_Hilbert,dtype=int)  ### |Na_r,Nb_r><Na_l,Nb_l|
	vB_l=np.zeros(dim_Hilbert,dtype=int)  ### --> |Na_r,Nb_r;Na_l,Nb_l>>

	#### We use a dictionary to store the labels of the states |Na_r,Nb_r;Na_l,Nb_l>>
	labels={}

	k=0
	###### We got 2 general catergories for |Na_r,Nb_r;Na_l,Nb_l>>
	for na_r in range(0,N1+1):
		nb_r=N1-na_r
		for na_l in range(0,N2+1): #### The first is Na_r+Nb_r=N1; Na_l+Nb_l=N2
		    nb_l=N2-na_l
		    vA_r[k]=na_r
		    vB_r[k]=nb_r
		    vA_l[k]=na_l
		    vB_l[k]=nb_l
		    key=str(na_r)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		    #print(key)
		    labels[key]=k
		    k=k+1


	for na_r in range(0,N2+1):
		nb_r=N2-na_r
		for na_l in range(0,N1+1): #### The second is Na_r+Nb_r=N2; Na_l+Nb_l=N1
		    nb_l=N1-na_l
		    vA_r[k]=na_r
		    vB_r[k]=nb_r
		    vA_l[k]=na_l
		    vB_l[k]=nb_l
		    key=str(na_r)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		    #print(key)
		    labels[key]=k
		    k=k+1


	#print(vA_r)
	###### We proceed to create the Liouvillian matrix and operators
	for k2 in range(0,dim_Hilbert): ########### CAREFUL INDICES: k2 is the receiver, k1 is the origin.


		na_r=vA_r[k2]
		nb_r=vB_r[k2]
		na_l=vA_l[k2]
		nb_l=vB_l[k2]

		###################### First we create the Liouvillian term by term
		######################
		####### Exchange term:
		key1=str(na_r-1)+","+str(nb_r+1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]  ## Identify the state to which we are coupling
		    liouv_op[k2,k1]+=-0.5j*w*np.sqrt((na_r)*(nb_r+1)) ## Set up the matrix element

		key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=-0.5j*w*np.sqrt((na_r+1)*(nb_r))

		key1=str(na_r)+","+str(nb_r)+","+str(na_l+1)+","+str(nb_l-1)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=+0.5j*w*np.sqrt((na_l+1)*(nb_l))

		key1=str(na_r)+","+str(nb_r)+","+str(na_l-1)+","+str(nb_l+1)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=+0.5j*w*np.sqrt((na_l)*(nb_l+1))

		######################
		####### Interaction term:
		####### This term is diagonal, so it is easy:
		liouv_op[k2,k2]+=-1j*(U_S)*(na_r*(na_r-1)+nb_r*(nb_r-1))  #####
		liouv_op[k2,k2]+=+1j*(U_S)*(na_l*(na_l-1)+nb_l*(nb_l-1))

		######################
		####### First dissipator: (the one proportional to n_th+1)
		key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l+1)+","+str(nb_l-1)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=(1+n_th)*(kap_S)*np.sqrt((na_r+1)*nb_r*(na_l+1)*nb_l)

		liouv_op[k2,k2]+=-0.5*(1+n_th)*(kap_S)*(na_r*(nb_r+1))
		liouv_op[k2,k2]+=-0.5*(1+n_th)*(kap_S)*(na_l*(nb_l+1))

		######################
		####### Second dissipator: (the one proportional to n_th)
		key1=str(na_r-1)+","+str(nb_r+1)+","+str(na_l-1)+","+str(nb_l+1)
		if key1 in labels:
		    k1=labels[key1]
		    liouv_op[k2,k1]+=(n_th)*(kap_S)*np.sqrt((nb_r+1)*na_r*(nb_l+1)*na_l)

		liouv_op[k2,k2]+=-0.5*(n_th)*(kap_S)*(nb_r*(na_r+1))
		liouv_op[k2,k2]+=-0.5*(n_th)*(kap_S)*(nb_l*(na_l+1))


		###################### We now create some useful operators:
		######################

		####### na_op|Na_r,Nb_r><Na_l,Nb_l|=Na_r|Na_r,Nb_r><Na_l,Nb_l|
		na_op[k2,k2]+=na_r

		####### nb_op|Na_r,Nb_r><Na_l,Nb_l|=Nb_r|Na_r,Nb_r><Na_l,Nb_l|
		nb_op[k2,k2]+=nb_r

		####### a_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Na_r)|Na_r-1,Nb_r><Na_l,Nb_l|
		key1=str(na_r+1)+","+str(nb_r)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    a_op[k2,k1]+=np.sqrt(na_r+1)

		####### b_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Nb_r)|Na_r,Nb_r-1><Na_l,Nb_l|
		key1=str(na_r)+","+str(nb_r+1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    b_op[k2,k1]+=np.sqrt(nb_r+1)

		####### bda_op|Na_r,Nb_r><Na_l,Nb_l|=sqrt(Na_r*(Nb_r+1))|Na_r-1,Nb_r+1><Na_l,Nb_l|
		key1=str(na_r+1)+","+str(nb_r-1)+","+str(na_l)+","+str(nb_l)
		if key1 in labels:
		    k1=labels[key1]
		    bda_op[k2,k1]+=np.sqrt((na_r+1)*nb_r)

		####### Flat state: this is used to perform the trace in the vectorized formalism

		if na_r == na_l:
			if nb_r == nb_l:
				flat_state[k2]=1.0


	####################### We now convert the matrices to csc/csr format:
	if format2=='csc':
		liouv_op=sparse.lil_matrix.tocsc(liouv_op)
		a_op=sparse.lil_matrix.tocsc(a_op)
		b_op=sparse.lil_matrix.tocsc(b_op)
		na_op=sparse.lil_matrix.tocsc(na_op)
		nb_op=sparse.lil_matrix.tocsc(nb_op)
		bda_op=sparse.lil_matrix.tocsc(bda_op)
	elif format2=='csr':
		liouv_op=sparse.lil_matrix.tocsr(liouv_op)
		a_op=sparse.lil_matrix.tocsr(a_op)
		b_op=sparse.lil_matrix.tocsr(b_op)
		na_op=sparse.lil_matrix.tocsr(na_op)
		nb_op=sparse.lil_matrix.tocsr(nb_op)
		bda_op=sparse.lil_matrix.tocsr(bda_op)


	return liouv_op,flat_state,a_op,b_op,bda_op,na_op,nb_op


################### This function analyzes the BTC (plus U term) using Qutip
###################
def analysis_BTC_qutip(N,kap,U,w,n_th):
	jj_mat=N/2
	gam=kap/jj_mat
	u_s=U/jj_mat

	########### Collective operators ############

	Jx=qt.jmat(jj_mat,'x')
	Jy=qt.jmat(jj_mat,'y')
	Jz=qt.jmat(jj_mat,'z')
	Jp=qt.jmat(jj_mat,'+')
	Jm=qt.jmat(jj_mat,'-')
	idenJ=qt.qeye(Jz.shape[0])

	########### Liouvillian

	Liouv=qt.liouvillian(-w*Jx+2*u_s*(jj_mat*(jj_mat+1)*idenJ+Jz*Jz),[np.sqrt(gam*(n_th+1))*Jm,np.sqrt(gam*(n_th))*Jp])

	#rho_ss=qt.steadystate(Liouv)  #### We can compute the stationary state and observables
	#O_Sx=(rho_ss*Jx).tr()
	#O_Sy=(rho_ss*Jy).tr()
	#O_Sz=(rho_ss*Jz).tr()


	L_spectrum=sp.linalg.eigvals(Liouv.full()) #### We can compute the eigenvalues


	return L_spectrum  ##### We must specify what we return: L_spectrum, O_Sx, O_Sy, etc.




########################################
########################################
######################
###################### Example 1: comparison of Bosonic single sector
###################### with BTC single S spectra

# Producing data for Fig. 2(g)

In [ ]:
# Different system sizes are given by N_list
N_list=np.linspace(20,100,17)

# Find the L_{N,N} eigenspectra 
for N in N_list:
    print('N=',N)
    kap=1
    U=0.0*kap
    w=1.45*kap
    n_th=0
    #### We obtain the Liouvillian for the sectors (N1,N1),(N1,N2),(N2,N1),(N2,N2)
    time0=time.time()
    L,fs,a,b,bda,na,nb=bosonic_liouvillian_setup(int(N),kap,U,w,n_th)
    time1=time.time()
    print("creation time: ",time1-time0)
    
    L_dense=sp.sparse.csc_matrix.todense(L)
    time0=time.time() #### We diagonalize it (also eigenvectors)
    eigs=sp.linalg.eigvals(L_dense)
    np.savetxt('eigs_N_U00_'+str(int(N))+'.dat',eigs)
    time1=time.time()
    
    print("diag time: ",time1-time0)

    # Find the L_{N,N-1} eigenspectra 
for N in N_list:
    print('N=',N)
    N1=int(N)
    N2=int(N-1)
    kap=1
    U=0.0*kap
    w=1.45*kap
    n_th=0
    #### We obtain the Liouvillian for the sectors (N1,N1),(N1,N2),(N2,N1),(N2,N2)
    time0=time.time()
    L,fs,a,b,bda,na,nb=bosonic_liouvillian_coherences2sectors_setup(N1,N2,kap,U,w,n_th)
    time1=time.time()
    print("creation time: ",time1-time0)
    
    L_dense=sp.sparse.csc_matrix.todense(L)
    time0=time.time() #### We diagonalize it (also eigenvectors)
    eigs=sp.linalg.eigvals(L_dense)
    np.savetxt('BTC_N1_N2_'+str(int(N))+'.dat',eigs)
    time1=time.time()
    
    print("diag time: ",time1-time0)

# Producing data for Fig. 2(h)

In [ ]:
# Different system sizes are given by N_list
N_list=np.linspace(20,100,17)

# Find the L_{N,N} eigenspectra 
for N in N_list:
    print('N=',N)
    kap=1
    U=0.25*kap
    w=0.8*kap
    n_th=0
    #### We obtain the Liouvillian for the sectors (N1,N1),(N1,N2),(N2,N1),(N2,N2)
    time0=time.time()
    L,fs,a,b,bda,na,nb=bosonic_liouvillian_setup(int(N),kap,U,w,n_th)
    time1=time.time()
    print("creation time: ",time1-time0)
    
    L_dense=sp.sparse.csc_matrix.todense(L)
    time0=time.time() #### We diagonalize it (also eigenvectors)
    eigs=sp.linalg.eigvals(L_dense)
    np.savetxt('eigs_TC2_N_'+str(int(N))+'.dat',eigs)
    time1=time.time()
    
    print("diag time: ",time1-time0)

    # Find the L_{N,N-1} eigenspectra 
for N in N_list:
    print('N=',N)
    N1=int(N)
    N2=int(N-1)
    kap=1
    U=0.25*kap
    w=0.8*kap
    n_th=0
    #### We obtain the Liouvillian for the sectors (N1,N1),(N1,N2),(N2,N1),(N2,N2)
    time0=time.time()
    L,fs,a,b,bda,na,nb=bosonic_liouvillian_coherences2sectors_setup(N1,N2,kap,U,w,n_th)
    time1=time.time()
    print("creation time: ",time1-time0)
    
    L_dense=sp.sparse.csc_matrix.todense(L)
    time0=time.time() #### We diagonalize it (also eigenvectors)
    eigs=sp.linalg.eigvals(L_dense)
    np.savetxt('eigs_TC2_N1_N2_'+str(int(N))+'.dat',eigs)
    time1=time.time()
    
    print("diag time: ",time1-time0)

# Producing data for Fig. 2(i)

In [ ]:
# Different system sizes are given by N_list
N_list=np.linspace(20,100,17)

# Find the L_{N,N} eigenspectra 
for N in N_list:
    print('N=',N)
    kap=1
    U=0.25*kap
    w=1.45*kap
    n_th=0
    #### We obtain the Liouvillian for the sectors (N1,N1),(N1,N2),(N2,N1),(N2,N2)
    time0=time.time()
    L,fs,a,b,bda,na,nb=bosonic_liouvillian_setup(int(N),kap,U,w,n_th)
    time1=time.time()
    print("creation time: ",time1-time0)
    
    L_dense=sp.sparse.csc_matrix.todense(L)
    time0=time.time() #### We diagonalize it (also eigenvectors)
    eigs=sp.linalg.eigvals(L_dense)
    np.savetxt('igs_N_TCnew'+str(int(N))+'.dat',eigs)
    time1=time.time()
    
    print("diag time: ",time1-time0)

    # Find the L_{N,N-1} eigenspectra 
for N in N_list:
    print('N=',N)
    N1=int(N)
    N2=int(N-1)
    kap=1
    U=0.25*kap
    w=1.45*kap
    n_th=0
    #### We obtain the Liouvillian for the sectors (N1,N1),(N1,N2),(N2,N1),(N2,N2)
    time0=time.time()
    L,fs,a,b,bda,na,nb=bosonic_liouvillian_coherences2sectors_setup(N1,N2,kap,U,w,n_th)
    time1=time.time()
    print("creation time: ",time1-time0)
    
    L_dense=sp.sparse.csc_matrix.todense(L)
    time0=time.time() #### We diagonalize it (also eigenvectors)
    eigs=sp.linalg.eigvals(L_dense)
    np.savetxt('eigs_N1_N2_TCnew'+str(int(N))+'.dat',eigs)
    time1=time.time()
    
    print("diag time: ",time1-time0)